# ERNIE-4.5-VL-28B-A3B-Paddle本地化部署与调用教程

## 文心开源模型的选择

### ERNIE-4.5模型系列规格对比表

| 模型系列 | 模型名称 | 总参数 | 激活参数 | 模态支持 | 上下文长度 | 主要用途 | 部署场景 |
|---------|---------|--------|---------|---------|-----------|---------|---------|
| **A47B大规模** | ERNIE-4.5-300B-A47B-Base | 300B | 47B | 文本 | 128K | 预训练基座 | 云端GPU集群 |
| | ERNIE-4.5-300B-A47B | 300B | 47B | 文本 | 128K | 指令遵循/创意生成 | 云端GPU集群 |
| | ERNIE-4.5-VL-424B-A47B-Base | 424B | 47B | 文本+视觉 | 128K | 多模态预训练 | 云端GPU集群 |
| | ERNIE-4.5-VL-424B-A47B | 424B | 47B | 文本+视觉 | 128K | 图文理解/生成 | 云端GPU集群 |
| **A3B中等规模** | ERNIE-4.5-21B-A3B-Base | 21B | 3B | 文本 | 128K | 预训练基座 | 单机多卡 |
| | ERNIE-4.5-21B-A3B | 21B | 3B | 文本 | 128K | 对话/文档处理 | 单机多卡 |
| | ERNIE-4.5-VL-28B-A3B-Base | 28B | 3B | 文本+视觉 | 128K | 多模态预训练 | 单机多卡 |
| | ERNIE-4.5-VL-28B-A3B | 28B | 3B | 文本+视觉 | 128K | 轻量多模态应用 | 单机多卡 |
| **0.3B轻量** | ERNIE-4.5-0.3B-Base | 0.3B | 0.3B | 文本 | 4K | 端侧预训练 | 移动端/边缘 |
| | ERNIE-4.5-0.3B | 0.3B | 0.3B | 文本 | 4K | 实时对话 | 移动端/边缘 |

### 模型规格选择策略表

| 应用场景 | 推荐模型 | 理由 | 硬件要求 | 推理延迟 |
|---------|---------|------|---------|---------|
| **复杂推理任务** | ERNIE-4.5-300B-A47B | 最强推理能力 | 8×A100(80GB) | 高 |
| **创意内容生成** | ERNIE-4.5-300B-A47B | 最佳创意表现 | 8×A100(80GB) | 高 |
| **多模态理解** | ERNIE-4.5-VL-424B-A47B | 图文融合理解 | 8×A100(80GB) | 高 |
| **日常对话客服** | ERNIE-4.5-21B-A3B | 性能成本平衡 | 4×V100(32GB) | 中 |
| **文档信息抽取** | ERNIE-4.5-21B-A3B | 理解能力充足 | 4×V100(32GB) | 中 |
| **轻量多模态** | ERNIE-4.5-VL-28B-A3B | 图文处理均衡 | 4×V100(32GB) | 中 |
| **移动端应用** | ERNIE-4.5-0.3B | 低延迟快响应 | 1×GPU/CPU | 低 |
| **边缘计算** | ERNIE-4.5-0.3B | 资源消耗最小 | CPU/NPU | 低 |

### 为什么选择ERNIE-4.5-VL-28B-A3B-Paddle模型？

#### 1. **性能与成本的最佳平衡**
- **参数规模适中**：28B总参数，3B激活参数，在保证推理能力的同时控制了计算成本
- **硬件要求合理**：支持单机多卡部署（4×V100或2×A100），相比A47B系列降低了75%的硬件门槛
- **推理延迟适中**：在保证输出质量的前提下，响应速度比大规模模型快3-5倍

#### 2. **多模态能力突出**
- **文本+视觉融合**：原生支持图文理解，无需额外的视觉编码器
- **长上下文支持**：128K token上下文长度，可处理长文档和多张图片
- **应用场景丰富**：适合文档分析、图像描述、多模态问答等任务

#### 3. **部署友好性**
- **AIStudio平台优化**：官方深度适配，提供一键下载和部署
- **FastDeploy集成**：完整的推理加速和服务化支持
- **开源生态**：PaddlePaddle生态系统，文档完善，社区活跃

#### 4. **实际应用价值**
- **企业级可用**：相比0.3B模型，理解能力和生成质量显著提升
- **成本可控**：相比A47B系列，部署成本降低70%，运营成本降低60%
- **扩展性强**：支持LoRA微调，可针对特定场景进行优化

#### 5. **技术先进性**
- **MoE架构**：专家混合模型，激活参数仅为总参数的1/9，推理效率高
- **多模态对齐**：视觉和文本特征深度融合，理解能力接近GPT-4V
- **中文优化**：针对中文场景深度优化，在中文多模态任务上表现优异

#### 选择建议
**推荐场景**：
- 中小企业多模态AI应用开发
- 教育科研项目的多模态实验
- 个人开发者的AI产品原型验证
- 需要图文理解能力的业务系统

**不推荐场景**：
- 对推理延迟要求极高的实时系统（选择0.3B）
- 预算充足且追求极致性能的场景（选择A47B）
- 纯文本应用且对多模态无需求（选择21B-A3B）

## 一、环境准备
### 1. 硬件要求
- **GPU**：NVIDIA A100 80GB（支持单卡/多卡，推荐CUDA 11.8+）  
- **内存**：≥60GB RAM  
- **存储**：≥120GB（模型约28GB，需预留日志/缓存空间）  

![](https://ai-studio-static-online.cdn.bcebos.com/a7c6679662a74f658efabd4e1fcbd612475536ce81ec45fea5442fa357938520)

### 2. 软件依赖

In [20]:
%%capture
!python -m pip install fastdeploy-gpu -i https://www.paddlepaddle.org.cn/packages/stable/fastdeploy-gpu-80_90/ --extra-index-url https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple

### 3. 环境验证
#### ① 检查CUDA版本

In [21]:
!nvcc --version
# 预期输出示例：

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Oct_29_23:50:19_PDT_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


#### ② 检查GPU信息

In [22]:
# 检查GPU状态和显存
!nvidia-smi
# 预期输出示例：

Sun Jul  6 18:10:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A800-SXM4-80GB          On  |   00000000:D0:00.0 Off |                    0 |
| N/A   29C    P0             64W /  400W |   68463MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

#### ③ 环境验证说明
| 检查项目 | 最低要求 | 推荐配置 | 说明 |
|---------|---------|---------|------|
| **CUDA版本** | 11.8+ | 12.6+ | 支持最新的GPU加速特性 |
| **驱动版本** | 520.61.05+ | 570.148.08+ | 兼容A800/A100系列GPU |
| **GPU型号** | V100(32GB) | A800/A100(80GB) | 确保足够的显存运行28B模型 |
| **显存容量** | 32GB+ | 80GB+ | 模型加载约需25-30GB显存 |
| **GPU利用率** | 0% | 0% | 启动前GPU应处于空闲状态 |

#### ④ 常见环境问题排查

In [28]:
# 检查PaddlePaddle GPU支持
!python -c "import paddle; print('GPU可用:', paddle.is_compiled_with_cuda()); print('GPU设备数:', paddle.device.cuda.device_count())"

# 检查FastDeploy安装
!python -c "from fastdeploy import LLM, SamplingParams; print('FastDeploy安装成功！')"

# 检查OpenAI库版本
!python -c "import openai; print('OpenAI库版本:', openai.__version__)"

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
GPU可用: True
GPU设备数: 1
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
FastDeploy安装成功！
OpenAI库版本: 1.91.0


## 二、模型下载与目录结构
### 1. 下载模型文件

In [19]:
# 使用AIStudio命令下载模型
%%capture
!aistudio download --model PaddlePaddle/ERNIE-4.5-VL-28B-A3B-Paddle --local_dir /home/aistudio/work/models

In [32]:
# 查看模型文件
!ls -l work/models

total 57441133
-rw-r--r-- 1 aistudio aistudio      11366 Jul  5 18:29 LICENSE
-rw-r--r-- 1 aistudio aistudio       9077 Jul  5 18:30 README.md
-rw-r--r-- 1 aistudio aistudio      86904 Jul  5 18:29 added_tokens.json
-rw-r--r-- 1 aistudio aistudio       1306 Jul  5 18:28 config.json
-rw-r--r-- 1 aistudio aistudio        134 Jul  5 18:29 generation_config.json
-rw-r--r-- 1 aistudio aistudio 4991326368 Jul  5 18:30 model-00001-of-00012.safetensors
-rw-r--r-- 1 aistudio aistudio 4988696384 Jul  5 18:29 model-00002-of-00012.safetensors
-rw-r--r-- 1 aistudio aistudio 4999185600 Jul  5 18:30 model-00003-of-00012.safetensors
-rw-r--r-- 1 aistudio aistudio 4995268296 Jul  5 18:29 model-00004-of-00012.safetensors
-rw-r--r-- 1 aistudio aistudio 4988696984 Jul  5 18:30 model-00005-of-00012.safetensors
-rw-r--r-- 1 aistudio aistudio 4999193256 Jul  5 18:29 model-00006-of-00012.safetensors
-rw-r--r-- 1 aistudio aistudio 4995261896 Jul  5 18:30 model-00007-of-00012.safetensors
-rw-r--r-- 1 aistudio a

### 2. 目录结构
```
work/
└── models/
    ├── LICENSE                           # 许可证文件
    ├── README.md                         # 模型说明文档
    ├── added_tokens.json                 # 新增token配置
    ├── config.json                       # 模型配置文件
    ├── generation_config.json            # 生成配置文件
    ├── model-00001-of-00012.safetensors  # 模型参数文件(分片1/12)
    ├── model-00002-of-00012.safetensors  # 模型参数文件(分片2/12)
    ├── model-00003-of-00012.safetensors  # 模型参数文件(分片3/12)
    ├── model-00004-of-00012.safetensors  # 模型参数文件(分片4/12)
    ├── model-00005-of-00012.safetensors  # 模型参数文件(分片5/12)
    ├── model-00006-of-00012.safetensors  # 模型参数文件(分片6/12)
    ├── model-00007-of-00012.safetensors  # 模型参数文件(分片7/12)
    ├── model-00008-of-00012.safetensors  # 模型参数文件(分片8/12)
    ├── model-00009-of-00012.safetensors  # 模型参数文件(分片9/12)
    ├── model-00010-of-00012.safetensors  # 模型参数文件(分片10/12)
    ├── model-00011-of-00012.safetensors  # 模型参数文件(分片11/12)
    ├── model-00012-of-00012.safetensors  # 模型参数文件(分片12/12)
    ├── model.safetensors.index.json      # 模型分片索引文件
    ├── preprocessor_config.json          # 预处理器配置
    ├── special_tokens_map.json           # 特殊token映射
    ├── tokenizer.model                   # 分词器模型文件
    └── tokenizer_config.json             # 分词器配置文件
```

## 三、启动服务（关键命令）

In [ ]:
!python -m fastdeploy.entrypoints.openai.api_server \
       --model work/models \
       --port 8180 \
       --metrics-port 8181 \
       --engine-worker-queue-port 8182 \
       --max-model-len 32768 \
       --enable-mm \
       --reasoning-parser ernie-45-vl \
       --max-num-seqs 32

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
/home/aistudio/external-libraries/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[2025-07-05 18:31:56,005] [    INFO] - loading configuration file work/models/preprocessor_config.json
INFO     2025-07-05 18:31:59,153 14427 engine.py[line:206] Waitting worker processes ready...
Loading Layers: 100%|█████████████████████████| 100/100 [00:07<00:00, 13.31it/s]
I

## 模型部署（推荐新建终端进行部署）
![](https://ai-studio-static-online.cdn.bcebos.com/adb145c8b36e4bfcb4eb4fc95addd20dedc26cffd6514e189c18900cbc593dc2)

### 服务启动参数详细说明

| 参数名称 | 参数说明 | 默认值 | 示例值 |
|---------|---------|-------|-------|
| `--model` | 模型文件路径，包含模型权重和配置文件的目录 | 必选 | `work/models` |
| `--port` | API服务监听端口，客户端通过此端口访问服务 | 8000 | `8180` |
| `--metrics-port` | 监控指标服务端口，用于性能监控和健康检查 | 8001 | `8181` |
| `--engine-worker-queue-port` | 引擎工作队列端口，用于内部任务调度 | 8002 | `8182` |
| `--max-model-len` | 模型最大支持的序列长度（token数） | 2048 | `32768` |
| `--enable-mm` | 启用多模态功能（文本+图像处理） | False | `开启` |
| `--reasoning-parser` | 推理解析器类型，指定模型的推理逻辑 | 无 | `ernie-45-vl` |
| `--max-num-seqs` | 最大并发序列数，控制批处理大小 | 256 | `32` |

### 其他常用参数

| 参数名称 | 参数说明 | 默认值 | 备注 |
|---------|---------|-------|------|
| `--host` | 服务绑定的主机地址 | 0.0.0.0 | 设置为0.0.0.0允许外部访问 |
| `--trust-remote-code` | 信任远程代码执行 | False | 加载自定义模型时需要 |
| `--tensor-parallel-size` | 张量并行大小（多GPU） | 1 | 根据GPU数量设置 |
| `--gpu-memory-utilization` | GPU内存利用率 | 0.9 | 建议0.8-0.95之间 |
| `--max-num-batched-tokens` | 最大批处理token数 | 自动计算 | 根据GPU显存调整 |
| `--swap-space` | 交换空间大小(GB) | 4 | 内存不足时使用 |
| `--enable-lora` | 启用LoRA适配器 | False | 微调模型时使用 |
| `--max-log-len` | 最大日志长度 | 无限制 | 控制日志文件大小 |

### 服务验证

In [43]:
# 检查模型是否正常加载
!curl -X POST http://localhost:8180/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{"model": "null","messages": [{"role": "user", "content": "你好，请介绍一下你自己"}],"temperature": 0.7}'

{"id":"chatcmpl-bd47b6ab-f962-4ad2-bf8c-e418cad02b8e","object":"chat.completion","created":1751798020,"model":"null","choices":[{"index":0,"message":{"role":"assistant","content":"\n\n你好呀！我是你的智能助手，可以帮你解答各种问题、提供建议，或者陪你聊聊各种话题。无论是学习上的疑问、生活中的小问题，还是想听个故事、写点创意内容，我都能陪你一起探索！随时等你来互动哦~ 😊","reasoning_content":"\n用户让我介绍一下自己，我需要用自然、友好的方式回应。首先，得确定用户的需求是什么。可能他们刚接触这个平台，想了解AI的基本功能，或者只是好奇我的能力。作为AI助手，我应该突出自己的主要功能，比如回答问题、提供建议、帮助学习等。\n\n接下来，要考虑用户的潜在需求。他们可能想知道我能做什么，或者如何与我互动。所以，我需要列举几个具体的例子，比如解释复杂概念、生成创意内容、处理日常任务等，这样用户能更直观地理解我的用途。\n\n还要注意语气要亲切，避免使用过于技术化的术语，让用户感觉轻松。比如用“随时陪你聊天”而不是“提供24/7支持”。同时，保持回答简洁，但信息量足够，让用户有进一步探索的欲望。\n\n另外，用户可能没有明确说出他们的具体需求，所以需要覆盖不同的方面，比如学习、工作、娱乐等，展示我的多功能性。比如提到写诗、做计划、翻译，这样用户可以根据自己的情况选择使用场景。\n\n最后，结尾部分要鼓励用户提问，让他们知道可以随时开始对话，保持开放的态度。这样整个回答既全面又友好，符合用户的需求和期望。\n","tool_calls":null},"finish_reason":"stop"}],"usage":{"prompt_tokens":14,"total_tokens":331,"completion_tokens":317,"prompt_tokens_details":{"cached_tokens":0}}}

## 四、模型调用示例

### 1. OpenAI库调用方式

#### ① 文本生成

In [5]:
import openai
host = "0.0.0.0"
port = "8180"
client = openai.Client(base_url=f"http://{host}:{port}/v1", api_key="null")

response = client.chat.completions.create(
    model="null",
    messages=[
        {"role": "user", "content": "你是Aistudio和文心大模型开发的智能助手，请介绍一下你自己."}
    ],
    stream=True,
)
for chunk in response:
    if chunk.choices[0].delta:
        print(chunk.choices[0].delta.content, end='')



我是由 **Aistudio**（百度AI技术生态平台）与 **文心大模型** 技术体系共同赋能的智能助手。以下是我的核心特点与功能介绍：

### 1. **技术背景**
   - 基于百度自主研发的 **文心大模型** 技术，融合多模态理解、深度逻辑推理与自然语言生成能力。
   - 依托 **Aistudio** 的开发工具与生态资源，支持快速接入AI能力进行创新应用。

### 2. **核心能力**
   - **知识问答**：覆盖广泛领域（科技、文化、生活等），提供准确、结构化的回答。
   - **文本生成**：支持创意写作、文案生成、代码辅助、摘要提炼等多样化文本创作。
   - **逻辑推理**：处理复杂问题，分析因果关系，辅助决策与问题拆解。
   - **多轮对话**：支持上下文理解，实现流畅的连续交互体验。

### 3. **应用场景**
   - **教育科研**：辅助学习、论文辅助、知识图谱构建。
   - **创作辅助**：故事生成、诗歌创作、营销文案优化。
   - **日常助手**：日程管理、信息查询、生活建议。
   - **开发支持**：API调用、SDK集成，助力开发者快速构建AI应用。

### 4. **优势特点**
   - **高效精准**：快速响应复杂需求，减少信息筛选成本。
   - **持续学习**：通过用户反馈与迭代优化，不断提升能力边界。
   - **安全可靠**：遵循隐私保护原则，确保交互内容合规安全。

### 5. **如何使用**
   - **直接对话**：通过网页、APP或集成平台与我交互。
   - **API/SDK接入**：开发者可调用接口实现定制化功能。
   - **探索案例**：访问Aistudio官网或相关文档，获取实战教程。

期待与你一起探索AI的无限可能！你可以尝试提问或提出需求，我会尽力提供支持。 😊

#### ② 图像描述生成

In [44]:
import openai
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

client = openai.Client(
    base_url="http://0.0.0.0:8180/v1",
    api_key="null"
)

response = client.chat.completions.create(
    model="null",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image('1.jpg')}"
                    }
                },
                {
                    "type": "text",
                    "text": "生成这张图片的描述"
                }
            ]
        }
    ],
    stream=True,
)

print("图片描述：", end='', flush=True)
for chunk in response:
    if chunk.choices and chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end='', flush=True)
print()

图片描述：
这张图片展示了飞桨自然语言处理开发库（PaddleNLP）的核心架构和功能模块，整体设计简洁清晰，采用蓝白配色，突出技术专业性。图片分为三个主要部分：

### 1. **工业级预置任务（Taskflow）**
   - **自然语言理解**：提供词法分析、文本纠错、情感分析、句法分析等基础理解能力。
   - **自然语言生成**：涵盖自动对联、智能写诗、生成式问答、开放域对话等应用场景，覆盖文本创作与交互需求。

### 2. **产业级模型库**
   - **自研预训练模型**：包含ERNIE系列（如ERNIE-1.0、ERNIE-2.0、ERNIE-Tiny等）、PLATO-2、SKEP等模型，支持多任务学习与小样本场景。
   - **覆盖全场景应用**：提供文本分类、匹配、生成、语义索引、信息抽取、文本翻译、知识关联等实用功能，适配多样化业务需求。

### 3. **文本领域核心API**
   - 提供数据管理（Data/Datasets）、嵌入表示（Embedding）、模型架构（Transformers/Seq2Vec）、评估指标（Metrics）及损失函数（Losses）等底层工具，支持开发者灵活构建自定义NLP应用。

### 底部标识
飞桨（PaddlePaddle）品牌标志强化了开源生态定位，整体架构体现了从预置任务到模型库再到开发工具的完整技术栈，适合企业级与开发者快速落地NLP场景。


### 2. requests库调用方式

#### ① 文本生成

In [17]:
import requests

url = "http://localhost:8180/v1/chat/completions"
headers = {"Content-Type": "application/json"}
payload = {
    "model": "null",
    "messages": [
        {"role": "user", "content": "你是Aistudio和文心大模型开发的智能助手，请介绍一下你自己."}
    ],
    "stream": True
}

response = requests.post(url, headers=headers, json=payload, stream=True)

for line in response.iter_lines():
    if line:
        line = line.decode('utf-8').replace('data: ', '')

        if line.strip() == '[DONE]':
            continue
        try:

            data = json.loads(line)

            if 'choices' in data and len(data['choices']) > 0:
                delta = data['choices'][0].get('delta', {})
                content = delta.get('content', '')
                if content:
                    print(content, end='', flush=True)
        except json.JSONDecodeError as e:
            print(f"Error decoding line: {line}")
            print(f"Error: {e}")

print()



我是由**Aistudio**（百度开源的深度学习平台）与**文心大模型**技术框架共同赋能的智能助手。以下是我的核心特点与功能介绍：

### 1. **身份背景**
   - 基于文心大模型的技术底座，融合了海量多领域知识库与深度学习算法，具备强大的语言理解与生成能力。
   - 由百度AI团队持续优化，注重逻辑推理、知识整合与交互体验的平衡。

### 2. **核心能力**
   - **自然语言交互**：支持中文及多语言对话，能理解复杂问题并给出结构化回答。
   - **知识问答**：涵盖科学、技术、人文、生活等广泛领域，提供精准信息与解释。
   - **文本生成**：可撰写代码、文章、诗歌、报告等，支持创意与实用场景需求。
   - **逻辑推理**：处理数学题、逻辑谜题、决策分析等需要推理的任务。
   - **学习辅助**：提供学习规划、知识点解析、语言学习等支持。

### 3. **典型应用场景**
   - **编程开发**：代码生成、调试建议、技术文档辅助。
   - **内容创作**：文案撰写、故事构思、学术写作支持。
   - **学习辅导**：解答学科问题、学习策略建议。
   - **日常助手**：信息查询、日程提醒、生活建议。

### 4. **优势特点**
   - **高效精准**：快速响应复杂需求，减少信息筛选成本。
   - **持续进化**：通过用户反馈与数据迭代，不断提升能力边界。
   - **安全可靠**：遵循伦理规范，注重隐私保护与内容合规。

### 5. **如何使用我？**
   - 直接提问或描述需求（如“写一篇关于AI伦理的议论文”）。
   - 指定任务类型（如“生成Python爬虫代码”）。
   - 提供上下文背景，帮助我理解更复杂的场景。

期待与你一起探索知识边界，解决实际问题！ 😊


#### ② 图像描述生成

In [18]:
import requests

url = "http://localhost:8180/v1/chat/completions"
payload = {
    "model":"null",
    "messages":[
        {
            "role":"user",
            "content":[
                {"type":"image_url","image_url":{"url":f"data:image/jpeg;base64,{encode_image('1.jpg')}"}},
                {"type":"text","text":"生成图片描述"}
            ]
        }
    ],
    "stream": True
}

response = requests.post(url, headers=headers, json=payload, stream=True)

for line in response.iter_lines():
    if line:
        line = line.decode('utf-8').replace('data: ', '')

        if line.strip() == '[DONE]':
            continue
        try:

            data = json.loads(line)

            if 'choices' in data and len(data['choices']) > 0:
                delta = data['choices'][0].get('delta', {})
                content = delta.get('content', '')
                if content:
                    print(content, end='', flush=True)
        except json.JSONDecodeError as e:
            print(f"Error decoding line: {line}")
            print(f"Error: {e}")

print()


该图片展示了飞桨自然语言处理开发库（PaddleNLP）的核心架构与功能模块，整体采用简洁的蓝白配色，结构清晰，分为三个主要层级：

### **1. 工业级预置任务（Taskflow）**
- **自然语言理解**：提供基础理解能力，包括词法分析（分词、词性标注等）、文本纠错、情感分析、句法分析（语法结构解析）。
- **自然语言生成**：覆盖多样化生成场景，如自动对联、智能写诗、生成式问答、开放域对话，支持创意性文本生成。

### **2. 产业级模型库**
- **自研预训练模型**：展示百度研发的系列模型，如ERNIE（含1.0/2.0/Tiny/Gram等版本）、PLATO（对话模型）、SKEP（情感倾向分析）、ERNIE-UNIMO（多模态理解）等，覆盖不同规模与任务需求。
- **全场景应用**：提供文本分类、匹配、生成、语义索引、小样本学习、文本图学习、信息抽取、翻译、知识关联、模型压缩等实用功能，适配金融、医疗、教育等多领域。

### **3. 文本领域核心API**
- **底层工具支持**：提供数据加载（Data/Datasets）、嵌入表示（Embedding）、Transformer框架（Transformers）、序列到向量转换（Seq2Vec）、评估指标（Metrics）及损失函数（Losses），为开发者提供灵活的模型构建与训练基础。

### **设计亮点**
- **模块化分层**：从任务到模型再到工具，形成完整的技术栈，降低开发门槛。
- **场景覆盖广**：兼顾通用理解与生成能力，同时支持垂直领域的高效应用。
- **技术生态完整**：通过核心API与预训练模型结合，兼顾灵活性与性能。

该架构体现了PaddleNLP在自然语言处理领域的全栈能力，旨在为开发者提供从基础工具到高级应用的全方位支持。


### 3. 调用参数说明

#### 常用参数配置
```python
# 完整的调用参数示例
response = client.chat.completions.create(
    model="null",                    # 模型名称（固定值）
    messages=[...],                  # 消息列表
    stream=True,                     # 是否启用流式响应
    max_tokens=2048,                 # 最大生成token数
    temperature=0.7,                 # 温度参数(0.0-2.0)
    top_p=0.9,                      # 核采样参数(0.0-1.0)
    frequency_penalty=0.0,           # 频率惩罚(-2.0-2.0)
    presence_penalty=0.0,            # 存在惩罚(-2.0-2.0)
    stop=["<|endoftext|>"],         # 停止词列表
)
```

#### 参数详细说明
| 参数名称 | 类型 | 默认值 | 说明 |
|---------|------|--------|------|
| `model` | str | "null" | 模型名称，本地部署时固定为"null" |
| `messages` | list | 必选 | 对话消息列表，包含role和content |
| `stream` | bool | False | 是否启用流式响应 |
| `max_tokens` | int | 自动 | 最大生成token数量 |
| `temperature` | float | 1.0 | 控制随机性，值越高越随机 |
| `top_p` | float | 1.0 | 核采样，控制词汇选择范围 |
| `frequency_penalty` | float | 0.0 | 频率惩罚，减少重复内容 |
| `presence_penalty` | float | 0.0 | 存在惩罚，鼓励谈论新话题 |
| `stop` | list | None | 停止生成的字符串列表 |

## 五、常见问题与解决方案
### 1. 端口占用问题
```bash
# 查看端口占用进程
lsof -i:8180

# 终止进程（替换<PID>为实际进程号）
kill -9 <PID>
```

### 2. 模型加载失败
- **检查目录**：确认`work/models`下存在完整的模型文件（.pdparams/config.json/vocab.txt）  
- **解析器参数**：确保启动命令包含`--reasoning-parser ernie-45-vl`  
- **驱动版本**：NVIDIA驱动需≥520.61.05（支持A100）  

### 3. 流式响应异常
- 确保`stream=True`参数正确传递  
- 检查服务日志（`fastdeploy_server.log`）是否有内存/显存不足报错  

## 六、测试用例
### 1. 文本生成验证
- **输入**：`"总结本教程的核心步骤"`  
- **预期**：输出包含"环境配置"、"服务启动"、"接口调用"等关键词的连贯文本  

### 2. 图像描述验证
- **测试图片**：使用包含自然场景的jpg文件（如山水、人物活动）  
- **预期**：输出包含景物特征（如"蓝天白云下的绿色草原"）、动作描述（如"人物正在湖边散步"）的语句  

## 七、资源链接
- **FastDeploy文档**：https://www.paddlepaddle.org.cn/fastdeploy  
- **AIStudio平台**：https://aistudio.baidu.com/  
- **模型仓库**：https://huggingface.co/baidu/ERNIE-4.5-VL-28B-A3B-Paddle  